In [44]:
import csv
import io
import os
import time
import uuid # <-- NEW: Import uuid
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

In [45]:
# 1. Define your data
CSV_DATA = """Domain,Service,PDF_URL
Compute,ec2,https://docs.aws.amazon.com/pdfs/AWSEC2/latest/UserGuide/ec2-ug.pdf
Compute,lambda,https://docs.aws.amazon.com/pdfs/lambda/latest/dg/lambda-dg.pdf
Compute,ecs,https://docs.aws.amazon.com/pdfs/AmazonECS/latest/developerguide/ecs-dg.pdf
Compute,eks,https://docs.aws.amazon.com/pdfs/eks/latest/userguide/eks-ug.pdf
Compute,elastic-beanstalk,https://docs.aws.amazon.com/pdfs/elasticbeanstalk/latest/dg/awseb-dg.pdf
Compute,batch,https://docs.aws.amazon.com/pdfs/batch/latest/userguide/batch_user.pdf
Storage,s3,https://docs.aws.amazon.com/pdfs/AmazonS3/latest/userguide/s3-userguide.pdf
Storage,ebs,https://docs.aws.amazon.com/pdfs/ebs/latest/userguide/ebs-ug.pdf
Storage,efs,https://docs.aws.amazon.com/pdfs/efs/latest/ug/efs-ug.pdf
Storage,glacier,https://docs.aws.amazon.com/pdfs/amazonglacier/latest/dev/glacier-dg.pdf
Networking,vpc,https://docs.aws.amazon.com/pdfs/vpc/latest/userguide/vpc-ug.pdf
Networking,route53,https://docs.aws.amazon.com/pdfs/Route53/latest/DeveloperGuide/route53-dg.pdf
Networking,cloudfront,https://docs.aws.amazon.com/pdfs/AmazonCloudFront/latest/DeveloperGuide/AmazonCloudFront_DevGuide.pdf
Networking,api-gateway,https://docs.aws.amazon.com/pdfs/apigateway/latest/developerguide/apigateway-dg.pdf
Networking,elasticloadbalancing,https://docs.aws.amazon.com/pdfs/elasticloadbalancing/latest/userguide/elb-ug.pdf
Networking,application-load-balancer,https://docs.aws.amazon.com/pdfs/elasticloadbalancing/latest/application/elb-ag.pdf
Networking,network-load-balancer,https://docs.aws.amazon.com/pdfs/elasticloadbalancing/latest/network/elb-ng.pdf
Networking,gateway-load-balancer,https://docs.aws.amazon.com/pdfs/elasticloadbalancing/latest/gateway/elb-gateway.pdf
Security,iam,https://docs.aws.amazon.com/pdfs/IAM/latest/UserGuide/iam-ug.pdf
Security,kms,https://docs.aws.amazon.com/pdfs/kms/latest/developerguide/kms-dg.pdf
Security,secrets-manager,https://docs.aws.amazon.com/pdfs/secretsmanager/latest/userguide/secretsmanager-userguide.pdf
Security,cognito,https://docs.aws.amazon.com/pdfs/cognito/latest/developerguide/cognito-dg.pdf
Security,cloudtrail,https://docs.aws.amazon.com/pdfs/awscloudtrail/latest/userguide/awscloudtrail-ug.pdf
Database,rds,https://docs.aws.amazon.com/pdfs/AmazonRDS/latest/UserGuide/rds-ug.pdf
Database,dynamodb,https://docs.aws.amazon.com/pdfs/amazondynamodb/latest/developerguide/dynamodb-dg.pdf
Database,redshift,https://docs.aws.amazon.com/pdfs/redshift/latest/dg/redshift-dg.pdf
Database,elasticache,https://docs.aws.amazon.com/pdfs/AmazonElastiCache/latest/dg/redis-ug.pdf
Management,cloudwatch,https://docs.aws.amazon.com/pdfs/AmazonCloudWatch/latest/monitoring/acw-ug.pdf
Management,cloudformation,https://docs.aws.amazon.com/pdfs/AWSCloudFormation/latest/UserGuide/cfn-ug.pdf
Management,ssm,https://docs.aws.amazon.com/pdfs/systems-manager/latest/userguide/systems-manager-ug.pdf
Management,codepipeline,https://docs.aws.amazon.com/pdfs/codepipeline/latest/userguide/codepipeline-user.pdf
Management,codebuild,https://docs.aws.amazon.com/pdfs/codebuild/latest/userguide/codebuild-user.pdf
Management,codeartifact,https://docs.aws.amazon.com/pdfs/codeartifact/latest/ug/codeartifact-user.pdf
ApplicationIntegration,sqs,https://docs.aws.amazon.com/pdfs/AWSSimpleQueueService/latest/SQSDeveloperGuide/sqs-dg.pdf
ApplicationIntegration,sns,https://docs.aws.amazon.com/pdfs/sns/latest/dg/sns-dg.pdf
ApplicationIntegration,step-functions,https://docs.aws.amazon.com/pdfs/step-functions/latest/dg/step-functions-dg.pdf
ApplicationIntegration,eventbridge,https://docs.aws.amazon.com/pdfs/eventbridge/latest/userguide/user-guide.pdf
Analytics,quicksight,https://docs.aws.amazon.com/pdfs/quicksuite/latest/userguide/amazon-quicksuite-user.pdf
Analytics,athena,https://docs.aws.amazon.com/pdfs/athena/latest/ug/athena-ug.pdf
Analytics,glue,https://docs.aws.amazon.com/pdfs/glue/latest/dg/glue-dg.pdf
Analytics,emr,https://docs.aws.amazon.com/pdfs/emr/latest/ManagementGuide/emr-mgmt.pdf
Analytics,kinesis,https://docs.aws.amazon.com/pdfs/streams/latest/dev/kinesis-dg.pdf
Analytics,opensearch,https://docs.aws.amazon.com/pdfs/opensearch-service/latest/developerguide/opensearch-service-dg.pdf
Analytics,sagemaker,https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf
Analytics,lakeformation,https://docs.aws.amazon.com/pdfs/lake-formation/latest/dg/lake-formation-dg.pdf
Analytics,datapipeline,https://docs.aws.amazon.com/pdfs/datapipeline/latest/DeveloperGuide/datapipeline-dg.pdf
"""

In [46]:
# 2. Define constants
CHROMA_DB_PATH = "./chroma_db_AWSDocs"
EMBEDDING_MODEL_NAME = "nomic-embed-text" 
BATCH_SIZE = 50  
CHUNK_SIZE = 1000 
CHUNK_OVERLAP = 200

In [47]:
def parse_csv_data(csv_data):
    """Parses the in-memory CSV string into a list of dictionaries."""
    service_docs = []
    f = io.StringIO(csv_data)
    reader = csv.DictReader(f)
    for row in reader:
        service_docs.append(row)
    return service_docs

In [48]:
print("Initializing Ollama embeddings...")
try:
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
    embeddings.embed_query("Test embedding")
except Exception as e:
    print(f"Error connecting to Ollama. Is it running?")

Initializing Ollama embeddings...


2025-11-11 22:03:46,414 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [49]:
vector_store = Chroma(
    collection_name="AWSDocs",
    persist_directory=CHROMA_DB_PATH,
    embedding_function=embeddings
)

In [50]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [51]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

In [52]:
loader = DoclingLoader(
    file_path='https://docs.aws.amazon.com/pdfs/AWSEC2/latest/UserGuide/ec2-ug.pdf',
    export_type=ExportType.MARKDOWN,
)

In [53]:
docs_as_markdown = loader.load()

2025-11-11 22:03:55,758 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 22:03:56,100 - INFO - Going to convert document batch...
2025-11-11 22:03:56,101 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 22:03:56,106 - INFO - Auto OCR model selected ocrmac.
2025-11-11 22:03:56,107 - INFO - Accelerator device: 'mps'
2025-11-11 22:04:08,284 - INFO - Accelerator device: 'mps'
2025-11-11 22:04:09,793 - INFO - Processing document ec2-ug.pdf
2025-11-11 22:28:52,930 - INFO - Finished converting document ec2-ug.pdf in 1500.82 sec.


In [55]:
markdown_content = docs_as_markdown[0].page_content

In [56]:
semantic_chunks = markdown_splitter.split_text(markdown_content)


In [57]:
for chunk in semantic_chunks:
    chunk.metadata["domain"] = 'domain'
    chunk.metadata["service"] = 'service_name'
    chunk.metadata["source"] = 'url'
final_chunks = text_splitter.split_documents(semantic_chunks)
if final_chunks:
    print(f"Found {len(final_chunks)} final chunks to add.")    

Found 10887 final chunks to add.


In [58]:
chunk_ids = [str(uuid.uuid4()) for _ in final_chunks]

In [ ]:
import re # Make sure to import re at the top of your script

# Loop through all chunks
for i in range(0, len(final_chunks)):
    
    # Get the current chunk
    chunk = final_chunks[i]
    
    h1 = chunk.metadata.get("Header 1", "")
    h2 = chunk.metadata.get("Header 2", "")
    h3 = chunk.metadata.get("Header 3", "")
    
    is_toc = 'table of contents' in h1.lower() or \
             'table of contents' in h2.lower() or \
             'table of contents' in h3.lower()

    content_to_check = chunk.page_content.strip() # Remove leading/trailing spaces
    is_table_junk = False
    if content_to_check: # Only check if the string is not empty
        is_table_junk = bool(re.fullmatch(r'[|\-\s]+', content_to_check))
    
    
    if is_toc or is_table_junk:
        if is_toc:
            print(f"Skipping chunk {i+1} (Table of Contents)")
        if is_table_junk:
            print(f"Skipping chunk {i+1} (Markdown table fragment)")
        pass # Skip this chunk
    
    else:
        batch_docs = [chunk] # Get the chunk as a list
        batch_ids = [chunk_ids[i]] # Get the corresponding ID
        
        batch_num = i + 1
        total_batches = len(final_chunks)

        print(f"  Adding batch {batch_num}/{total_batches}...")
        
        try:
            vector_store.add_documents(
                documents=batch_docs,
                ids=batch_ids
            )
        except Exception as e:
            print(f"Error adding batch {batch_num}/{total_batches}: {e}")
            continue 

  Adding batch 1/10887...


2025-11-11 22:41:04,655 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:04,750 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:04,795 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:04,860 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:04,895 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 2/10887...
  Adding batch 3/10887...
Skipping chunk 4 (Table of Contents)
Skipping chunk 5 (Table of Contents)
Skipping chunk 6 (Table of Contents)
Skipping chunk 7 (Table of Contents)
Skipping chunk 8 (Table of Contents)
Skipping chunk 9 (Table of Contents)
Skipping chunk 10 (Table of Contents)
Skipping chunk 11 (Table of Contents)
Skipping chunk 12 (Table of Contents)
Skipping chunk 13 (Table of Contents)
Skipping chunk 14 (Table of Contents)
Skipping chunk 15 (Table of Contents)
Skipping chunk 16 (Table of Contents)
Skipping chunk 17 (Table of Contents)
Skipping chunk 18 (Table of Contents)
Skipping chunk 19 (Table of Contents)
Skipping chunk 20 (Table of Contents)
Skipping chunk 21 (Table of Contents)
Skipping chunk 22 (Table of Contents)
Skipping chunk 23 (Table of Contents)
Skipping chunk 24 (Table of Contents)
Skipping chunk 25 (Table of Contents)
Skipping chunk 26 (Table of Contents)
Skipping chunk 27 (Table of Contents)
Skipping chunk 28 (Table of Contents)
Skip

2025-11-11 22:41:04,945 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:04,976 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:05,008 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:05,032 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:05,055 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:05,084 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:41:05,127 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 176/10887...
  Adding batch 177/10887...
  Adding batch 178/10887...
  Adding batch 179/10887...
  Adding batch 180/10887...
  Adding batch 181/10887...
  Adding batch 182/10887...


...

  Adding batch 10881/10887...
  Adding batch 10882/10887...
  Adding batch 10883/10887...


2025-11-11 22:51:49,280 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:51:49,361 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:51:49,476 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 10884/10887...
  Adding batch 10885/10887...
  Adding batch 10886/10887...


2025-11-11 22:51:49,528 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 22:51:49,591 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


  Adding batch 10887/10887...


In [ ]:
for i, service in enumerate(service_docs)

In [27]:
import csv
import io
import os
import time
import uuid # <-- NEW: Import uuid
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# 1. Define your data
CSV_DATA = """Domain,Service,PDF_URL
Analytics,sagemaker,https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf
"""

# 2. Define constants
CHROMA_DB_PATH = "./chroma_db_cloudyintel"
EMBEDDING_MODEL_NAME = "nomic-embed-text" 
BATCH_SIZE = 1  
CHUNK_SIZE = 2000 # Safety net: Max characters per *final* chunk
CHUNK_OVERLAP = 200

def parse_csv_data(csv_data):
    """Parses the in-memory CSV string into a list of dictionaries."""
    service_docs = []
    f = io.StringIO(csv_data)
    reader = csv.DictReader(f)
    for row in reader:
        service_docs.append(row)
    return service_docs

def load_and_process_pdfs(service_docs):
    """
    Downloads, loads, splits, and embeds all PDFs using Docling,
    then saves them to Chroma.
    """
    
    # 3. Initialize components
    
    # Embedding model (Using Ollama)
    print("Initializing Ollama embeddings...")
    try:
        embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
        embeddings.embed_query("Test embedding")
    except Exception as e:
        print(f"Error connecting to Ollama. Is it running?")
        return

    # Chroma vector store
    vector_store = Chroma(
        persist_directory=CHROMA_DB_PATH,
        embedding_function=embeddings
    )
    
    # Safety net splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    
    print(f"Initialized Chroma DB at {CHROMA_DB_PATH}")
    
    # 4. Process each service document
    total_services = len(service_docs)
    for i, service in enumerate(service_docs):
        domain = service['Domain']
        service_name = service['Service']
        url = service['PDF_URL']
        
        print(f"\n--- Processing {i+1}/{total_services}: {domain} - {service_name} ---")
        print(f"URL: {url}")
        
        try:
            start_time = time.time()
            
            # A. Load PDF using Docling's default AI chunker
            print("Initializing DoclingLoader (using default AI chunking)...")
            loader = DoclingLoader(
                file_path=url,
                # NO export_type specified, so it uses default ExportType.DOC_CHUNKS
            )
            
            print("Loading and parsing with Docling (this may take a while)...")
            # 'docs' is now a list of AI-powered semantic chunks
            docs = loader.load()
            
            if not docs:
                print("Docling returned no content. Skipping.")
                continue
            
            # B. Add our custom metadata to each semantic chunk
            for doc in docs:
                doc.metadata["domain"] = domain
                doc.metadata["service"] = service_name
                doc.metadata["source"] = url

            # C. Stage 2 Split: Safety net
            print(f"Applying safety split to {len(docs)} AI chunks...")
            final_chunks = text_splitter.split_documents(docs)
            
            # D. Embed and add final chunks to Chroma
            if final_chunks:
                print(f"Found {len(final_chunks)} final chunks to add.")
                
                # Create a list of unique IDs
                chunk_ids = [str(uuid.uuid4()) for _ in final_chunks]
                
                # Batch both the documents and the IDs
                for i in range(0, len(final_chunks), BATCH_SIZE):
                    batch_docs = final_chunks[i:i + BATCH_SIZE]
                    batch_ids = chunk_ids[i:i + BATCH_SIZE]
                    
                    batch_num = (i // BATCH_SIZE) + 1
                    total_batches = (len(final_chunks) // BATCH_SIZE) + 1

                    print(f"  Adding batch {batch_num}/{total_batches}...")
                    
                    vector_store.add_documents(
                        documents=batch_docs,
                        ids=batch_ids
                    )
                
                end_time = time.time()
                print(f"Successfully processed {service_name} in {end_time - start_time:.2f} seconds.")
            else:
                print(f"No text chunks extracted from {service_name}.")

        except Exception as e:
            print(f"Error: Failed to process {service_name} from {url}. Skipping. Details: {e}")

    print("\n--- All documents processed! ---")
    print(f"Vector database is persistent and saved in '{CHROMA_DB_PATH}'")
    
    return vector_store


In [24]:
OllamaEmbeddings(model=EMBEDDING_MODEL_NAME).embed_query("Initial connection test")
print("Ollama connection successful.")

2025-11-11 19:52:47,981 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Ollama connection successful.


In [ ]:
if os.path.exists(CHROMA_DB_PATH):
    print(f"Database already exists at {CHROMA_DB_PATH}.")
    print("To re-build, please delete this directory and run again.")
else:


Database already exists at ./chroma_db_AWSDocs.
To re-build, please delete this directory and run again.


In [28]:
service_docs = parse_csv_data(CSV_DATA)
load_and_process_pdfs(service_docs)

Initializing Ollama embeddings...


2025-11-11 19:53:43,159 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-11 19:53:43,182 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Initialized Chroma DB at ./chroma_db_cloudyintel

--- Processing 1/1: Analytics - sagemaker ---
URL: https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf
Initializing DoclingLoader (using default AI chunking)...
Loading and parsing with Docling (this may take a while)...


2025-11-11 19:53:45,772 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 19:53:45,834 - INFO - Going to convert document batch...
2025-11-11 19:53:45,837 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-11 19:53:45,847 - INFO - Auto OCR model selected ocrmac.
2025-11-11 19:53:45,849 - INFO - Accelerator device: 'mps'
2025-11-11 19:53:50,863 - INFO - Accelerator device: 'mps'
2025-11-11 19:53:52,338 - INFO - Processing document next-generation-sagemaker-ug.pdf
2025-11-11 19:54:28,595 - INFO - Finished converting document next-generation-sagemaker-ug.pdf in 44.15 sec.
Token indices sequence length is longer than the specified maximum sequence length for this model (3596 > 512). Running this sequence through the model will result in indexing errors


Applying safety split to 203 AI chunks...
Found 203 final chunks to add.
  Adding batch 1/204...


2025-11-11 19:54:29,815 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Error: Failed to process sagemaker from https://docs.aws.amazon.com/pdfs/next-generation-sagemaker/latest/userguide/next-generation-sagemaker-ug.pdf. Skipping. Details: Expected metadata value to be a str, int, float, bool, SparseVector, or None, got {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/0', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 9.27027010472973, 't': 782.0892856775794, 'r': 39.06756740202703, 'b': 771.5178571061508, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 3]}]}], 'origin': {'mimetype': 'application/pdf', 'binary_hash': 16562198080799201207, 'filename': 'next-generation-sagemaker-ug.pdf'}} which is a dict in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

--- All documents processed! ---
Vector database is persistent and saved in './chro

In [ ]:

def main():
    print("Checking Ollama connection...")
    try:
        OllamaEmbeddings(model=EMBEDDING_MODEL_NAME).embed_query("Initial connection test")
        print("Ollama connection successful.")
    except Exception as e:
        print("Error: Could not connect to Ollama.")
        print("Please make sure the Ollama application is running and you have run:")
        print(f"ollama pull {EMBEDDING_MODEL_NAME}")
        return

    if os.path.exists(CHROMA_DB_PATH):
        print(f"Database already exists at {CHROMA_DB_PATH}.")
        print("To re-build, please delete this directory and run again.")
    else:
        service_docs = parse_csv_data(CSV_DATA)
        load_and_process_pdfs(service_docs)

if __name__ == "__main__":
    main()